# Tutorial #1: Interfacing w/ the Library

**Target Audience:** Design Engineers / Front-end User
**Learning Objectives**
1. Primary Functionality
    1. Selecting a component
    2. Meaning / notation of characteristics
    3. Finding geometrical parameters which best suits target characteristics
    4. Export to `.gds` or dictionary compatable with `QComponent(options=...)`
2. Secondary Functionality
    1. Seeing who has contributed what data
    2. Seeing what was the simulation method of the contributed data


In [5]:
from metal_library import Reader
reader = Reader(component_name='TransmonCross')
reader.config

{'qiskit-metal-version': '0.1.5',
 'qiskit-metal-component-path': 'qiskit_metal.qlibrary.qubits.transmon_cross.TransmonCross',
 'metal-library-component-path': None,
 'component-types': {'QubitOnly': {'characteristics': [{'column_name': 'Qubit_Frequency_GHz',
     'blurb': 'Qubit 01 Transition Freq (Linear GHz)',
     'units': 'GHz',
     'latex_symbol': 'f_{q,01}'},
    {'column_name': 'Qubit_Anharmonicity_MHz',
     'blurb': 'Qubit Anharmonicity (Linear MHz)',
     'units': 'MHz',
     'latex_symbol': '\\alpha_{q}'}]},
  'QubitCavity': {'characteristics': [{'column_name': 'Qubit_Frequency_GHz',
     'blurb': 'Qubit 01 Transition Freq (Linear GHz)',
     'units': 'GHz',
     'latex_symbol': 'f_{q,01}'},
    {'column_name': 'Qubit_Anharmonicity_MHz',
     'blurb': 'Qubit Anharmonicity (Linear MHz)',
     'units': 'MHz',
     'latex_symbol': '\\alpha_{q}'},
    {'column_name': 'Cavity_Frequency_GHz',
     'blurb': 'Cavity Resonant Freq (Linear MHz)',
     'units': 'GHz',
     'latex_sym